# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

## 1. What test is appropriate for this problem? Does CLT apply?

A bootstrap test for identical distributions is appropriate. The CLT does apply since the data is random and independent, and since the sample size n ≥ 30, the sample is large enough.

## 2. What are the null and alternate hypotheses?

The null hypothesis is that there is no significant difference between the calls received by white sounding names and black sounding names. 

The alternative hypothesis is that there is a significant difference between the calls received by the differing races. 

## 3.  Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [6]:
w_call = w.call
b_call = b.call
w_mean = np.mean(w_call)
b_mean = np.mean(b_call)
w_std = np.std(w_call)
b_std = np.std(b_call)

In [7]:
# Bootstrap approach
def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""
    diff = np.mean(data_1) - np.mean(data_2)
    return diff

empirical_diff_means = diff_of_means(w_call, b_call)
data_concat = np.concatenate((w_call, b_call))
bs_replicates = np.empty(10000)

for i in range(10000):
    # Generate bootstrap sample
    bs_sample = np.random.choice(data_concat, size=len(data_concat))
    
    # Compute replicate
    bs_replicates[i] = diff_of_means(bs_sample[:len(b_call)],
                                     bs_sample[len(b_call):])

# Compute and print p-value: p
p = np.sum(bs_replicates >= empirical_diff_means)/len(bs_replicates)
print('p-value =', p)

p-value = 0.0


The p-value is less than 0.05 (which is the alpha at a 95% confidence interval), and thus the null hypothesis can be rejected. Therefore, there is a significant difference between the calls received by different races.

In [8]:
# Frequentist approach: two sample t test
t, p = stats.ttest_ind(w_call, b_call, equal_var = False)
print('The t statistic is ', t, 'and the p value is ', p)

The t statistic is  4.114705290861751 and the p value is  3.942941513645935e-05


The p-value is less than 0.05 again, so the null hypothesis can be rejected. Thus, there is a significant difference between the calls received by white sounding names and black sounding names.

In [9]:
# Margin of error
z = 1.96
df = len(w_call) + len(b_call) - 2
std = np.sqrt(((len(w_call)-1)*(w_std)**2 + (len(b_call)-1)*(b_std)**2)/df)
error = z*std*np.sqrt((1/len(w_call))+(1/len(b_call)))
print('Margin of error = ', error)

Margin of error =  0.015255284385449893


In [10]:
# Confidence interval at 95%
ci = [empirical_diff_means - error, empirical_diff_means + error]
print('The 95% confidence interval is', ci)

The 95% confidence interval is [0.016777570469610682, 0.04728813924051047]


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

## 4. Write a story describing the statistical significance in the context or the original problem.

Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers. The statistical analysis shows a significant difference in the calls received by black-sounding names versus white-sounding names, thus the racism is still prevalent in the work force. 

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No the analysis does not indicate that the race associated with ones name is the most important factor in callback success. There are other variables that have not been analyzed, so there is no telling the impact of the name factor relative to other variables. I would ammend the analysis by not only testing the name, but also the other factors included in the dataset. 